# Загрузки

In [ ]:
!pip install openai

In [ ]:
!pip install transformers protobuf sentencepiece accelerate

# Задание 1

Используя OpenAI API попробуйте сделать следующие вещи:

1) Попробуйте воспроизвести частеречную разметку с помощью gpt-4o-mini. Для данного предложения попробуйте получить точно такую же разметку.
   Если не работает, то попробуйте добавить примеров из opencorpora (не добавляя этот пример). Если у вас сработало это предложение, то попробуйте любое другое и оцените насколько промпт обобщает.  
   !Не размечайте большое количество предложений в датасете и не рассчитывайте метрики качества, оценивайте только вручную

In [136]:
from openai import OpenAI
import json
import random
from google.colab import userdata

In [139]:
API = userdata.get("api_hse")

In [140]:
client = OpenAI(api_key=API)

In [ ]:
sentence = "Великолепная «Школа злословия» вернулась в эфир после летних каникул в новом формате."
sentence_pos = [['Великолепная', 'ADJF'],
 ['«', 'PNCT'],
 ['Школа', 'NOUN'],
 ['злословия', 'NOUN'],
 ['»', 'PNCT'],
 ['вернулась', 'VERB'],
 ['в', 'PREP'],
 ['эфир', 'NOUN'],
 ['после', 'PREP'],
 ['летних', 'ADJF'],
 ['каникул', 'NOUN'],
 ['в', 'PREP'],
 ['новом', 'ADJF'],
 ['формате', 'NOUN'],
 ['.', 'PNCT']]

In [ ]:
def form_messages(query, system_message=None):
    if system_message is not None:
        system = {"role": "system", "content": system_message}
        return [system, {"role": "user", "content": query}]
    return [{"role": "user", "content": query}]

def send_query(query, system_message=None, model_name="gpt-4o-mini"):
    response = client.chat.completions.create(
      model=model_name,
      messages=form_messages(query, system_message)
    )
    return response


## Zero-shot только с user-prompt

Ты — система морфологической разметки русского текста. Используй только следующие части речи и сокращения:

```
NOUN — имя существительное

ADJF — имя прилагательное (полное)

ADJS — имя прилагательное (краткое)

COMP — компаратив

VERB — глагол (личная форма)

INFN — глагол (инфинитив)

PRTF — причастие (полное)

PRTS — причастие (краткое)

GRND — деепричастие

NUMR — числительное

ADVB — наречие

NPRO — местоимение-существительное

PRED — предикатив

PREP — предлог

CONJ — союз

PRCL — частица

INTJ — междометие

PNCT — знак пунктуации
```

Разметь каждое слово в предложении соответствующим тегом.
Не добавляй других пояснений или описаний.

Формат вывода строго в json следующего формата:

[["слово", "тег"], ["слово", "тег"], ...]

Разметь следующее предложение: {sentence}

In [ ]:
user_prompt = """Ты — система морфологической разметки русского текста. Используй только следующие части речи и сокращения:

```
NOUN — имя существительное

ADJF — имя прилагательное (полное)

ADJS — имя прилагательное (краткое)

COMP — компаратив

VERB — глагол (личная форма)

INFN — глагол (инфинитив)

PRTF — причастие (полное)

PRTS — причастие (краткое)

GRND — деепричастие

NUMR — числительное

ADVB — наречие

NPRO — местоимение-существительное

PRED — предикатив

PREP — предлог

CONJ — союз

PRCL — частица

INTJ — междометие

PNCT — знак пунктуации
```

Разметь каждое слово в предложении соответствующим тегом.
Не добавляй других пояснений или описаний.

Формат вывода строго в json следующего формата:

[["слово", "тег"], ["слово", "тег"], ...]

Разметь следующее предложение: {sentence}"""

In [ ]:
prompt_1 = user_prompt.format(sentence=sentence)
response = send_query(prompt_1)

In [ ]:
for k, m in zip(sentence_pos, json.loads(response.choices[0].message.content)+ ['_']*len(sentence_pos)):
    print(f'Original: {str(k):<30} Auto-Annotation: {m}')

Original: ['Великолепная', 'ADJF']       Auto-Annotation: ['Великолепная', 'ADJF']
Original: ['«', 'PNCT']                  Auto-Annotation: ['«Школа', 'NOUN']
Original: ['Школа', 'NOUN']              Auto-Annotation: ['злословия»', 'NOUN']
Original: ['злословия', 'NOUN']          Auto-Annotation: ['вернулась', 'VERB']
Original: ['»', 'PNCT']                  Auto-Annotation: ['в', 'PREP']
Original: ['вернулась', 'VERB']          Auto-Annotation: ['эфир', 'NOUN']
Original: ['в', 'PREP']                  Auto-Annotation: ['после', 'PREP']
Original: ['эфир', 'NOUN']               Auto-Annotation: ['летних', 'ADJF']
Original: ['после', 'PREP']              Auto-Annotation: ['каникул', 'NOUN']
Original: ['летних', 'ADJF']             Auto-Annotation: ['в', 'PREP']
Original: ['каникул', 'NOUN']            Auto-Annotation: ['новом', 'ADJF']
Original: ['в', 'PREP']                  Auto-Annotation: ['формате', 'NOUN']
Original: ['новом', 'ADJF']              Auto-Annotation: ['.', 'PNCT']
Ori

Неожиданно все сработало практически с первого раза, единственное – надо подправить промпт для того, чтобы работала пунктуация.

In [ ]:
user_prompt_2 = r"""Ты — система морфологической разметки русского текста. Используй только следующие части речи и сокращения:

```
NOUN — имя существительное

ADJF — имя прилагательное (полное)

ADJS — имя прилагательное (краткое)

COMP — компаратив

VERB — глагол (личная форма)

INFN — глагол (инфинитив)

PRTF — причастие (полное)

PRTS — причастие (краткое)

GRND — деепричастие

NUMR — числительное

ADVB — наречие

NPRO — местоимение-существительное

PRED — предикатив

PREP — предлог

CONJ — союз

PRCL — частица

INTJ — междометие

PNCT — знак пунктуации
```

Разметь каждое слово в предложении соответствующим тегом.
Не добавляй других пояснений или описаний.
Перед разметкой сделай всю пунктуацию (!"#$%&\'()*+,-./:;<=>?@^_`~) отдельным токеном.

Формат вывода строго в json следующего формата:

[["слово", "тег"], ["слово", "тег"], ...]

Разметь следующее предложение: {sentence}"""

In [ ]:
prompt_2 = user_prompt_2.format(sentence=sentence)
response = send_query(prompt_2)

In [ ]:
for k, m in zip(sentence_pos, json.loads(response.choices[0].message.content)+ ['_']*len(sentence_pos)):
    print(f'Original: {str(k):<30} Auto-Annotation: {m}')

Original: ['Великолепная', 'ADJF']       Auto-Annotation: ['Великолепная', 'ADJF']
Original: ['«', 'PNCT']                  Auto-Annotation: ['«', 'PNCT']
Original: ['Школа', 'NOUN']              Auto-Annotation: ['Школа', 'NOUN']
Original: ['злословия', 'NOUN']          Auto-Annotation: ['злословия', 'NOUN']
Original: ['»', 'PNCT']                  Auto-Annotation: ['»', 'PNCT']
Original: ['вернулась', 'VERB']          Auto-Annotation: ['вернулась', 'VERB']
Original: ['в', 'PREP']                  Auto-Annotation: ['в', 'PREP']
Original: ['эфир', 'NOUN']               Auto-Annotation: ['эфир', 'NOUN']
Original: ['после', 'PREP']              Auto-Annotation: ['после', 'PREP']
Original: ['летних', 'ADJF']             Auto-Annotation: ['летних', 'ADJF']
Original: ['каникул', 'NOUN']            Auto-Annotation: ['каникул', 'NOUN']
Original: ['в', 'PREP']                  Auto-Annotation: ['в', 'PREP']
Original: ['новом', 'ADJF']              Auto-Annotation: ['новом', 'ADJF']
Original: [

Теперь все идеально для этого предложения. Попробуем, работает ли такой zero-shot промпт для других предложений.

## Дополнительные примеры

In [ ]:
import xml.etree.ElementTree as ET

path = "annot.opcorpora.no_ambig.xml"

tree = ET.parse(path)
root = tree.getroot()

sentences = []
tagged = []
for text in root.findall(".//text"):
    for sent in text.findall(".//sentence"):
        tokens = []
        pos_tags = []
        for token in sent.findall(".//token"):
            word = token.get("text")
            g = token.find(".//g[@v]")
            pos = g.get("v") if g is not None else "UNK"
            pos_tags.append(pos)
            tokens.append(word)
        if tokens:
            sentences.append("\t".join(tokens))
            tagged.append("\t".join(pos_tags))

In [ ]:
pairs = [x for x in zip(sentences, tagged)]

In [ ]:
random.sample(pairs, 3)

[('Также\tпаровозы\tотстраняли\tв\t1965\t(\t№\t1300\t,\t1301\t,\t1303\t,\t1316\t)\t,\t1966\t(\t№\t1302\t,\t1311\t,\t1312\t,\t1315\t,\t1318\t)\tи\tв\t1969\tгоду\t(\t№\t1304\t—\t1308\t,\t1313\t,\t1314\t,\t1317\t,\t1319\t)\t.',
  'PRCL\tNOUN\tVERB\tPREP\tNUMB\tPNCT\tSYMB\tNUMB\tPNCT\tNUMB\tPNCT\tNUMB\tPNCT\tNUMB\tPNCT\tPNCT\tNUMB\tPNCT\tSYMB\tNUMB\tPNCT\tNUMB\tPNCT\tNUMB\tPNCT\tNUMB\tPNCT\tNUMB\tPNCT\tCONJ\tPREP\tNUMB\tNOUN\tPNCT\tSYMB\tNUMB\tPNCT\tNUMB\tPNCT\tNUMB\tPNCT\tNUMB\tPNCT\tNUMB\tPNCT\tNUMB\tPNCT\tPNCT'),
 ('•\tБарбадос', 'PNCT\tNOUN'),
 ('Йоркширские\tдамы\t,\tможет\tбыть\t,\tтоже\tне\tпрочь\tбыли\tбы\tпредложить\tподобную\tнастойку\tсвоим\tгостям\t,\tно\tвоспитание\tне\tпозволяет\t.',
  'ADJF\tNOUN\tPNCT\tVERB\tINFN\tPNCT\tPRCL\tPRCL\tADVB\tVERB\tPRCL\tINFN\tADJF\tNOUN\tADJF\tNOUN\tPNCT\tCONJ\tNOUN\tPRCL\tVERB\tPNCT')]

In [ ]:
sent_1 = '—Я этого не говорил.'
tags = 'PNCT\tNPRO\tNPRO\tPRCL\tVERB\tPNCT'.split('\t')

In [ ]:
sent_tags = [
    ['—', 'PNCT'],
    ['Я', 'NPRO'],
    ['этого', 'NPRO'],
    ['не', 'PRCL'],
    ['говорил', 'VERB'],
    ['.', 'PNCT']
]

In [ ]:
prompt_3 = user_prompt_2.format(sentence=sent_1)
response = send_query(prompt_3)

In [ ]:
for k, m in zip(sent_tags, json.loads(response.choices[0].message.content)+ ['_']*len(sentence_pos)):
    print(f'Original: {str(k):<30} Auto-Annotation: {m}')

Original: ['—', 'PNCT']                  Auto-Annotation: ['—', 'PNCT']
Original: ['Я', 'NPRO']                  Auto-Annotation: ['Я', 'NPRO']
Original: ['этого', 'NPRO']              Auto-Annotation: ['этого', 'NPRO']
Original: ['не', 'PRCL']                 Auto-Annotation: ['не', 'PRCL']
Original: ['говорил', 'VERB']            Auto-Annotation: ['говорил', 'VERB']
Original: ['.', 'PNCT']                  Auto-Annotation: ['как', 'PNCT']


In [ ]:
sent_2 = 'В России вышла первая практическая книга «В социальных сетях. Twitter — «140 символов самовыражения», основанная на бизнес-кейсах российских компаний.'

In [ ]:
text = 'В России вышла первая практическая книга « В социальных сетях . Twitter — « 140 символов самовыражения » , основанная на бизнес-кейсах российских компаний .'.split()

In [ ]:
tags = 'PREP\tNOUN\tVERB\tADJF\tADJF\tNOUN\tPNCT\tPREP\tADJF\tNOUN\tPNCT\tLATN\tPNCT\tPNCT\tNUMB\tNOUN\tNOUN\tPNCT\tPNCT\tPRTF\tPREP\tNOUN\tADJF\tNOUN\tPNCT'.split('\t')

In [ ]:
sent_tags_2 = []
for word, tag in zip(text, tags):
    sent_tags_2.append([word, tag])

In [ ]:
prompt_4 = user_prompt_2.format(sentence=sent_2)
response = send_query(prompt_4)

In [ ]:
for k, m in zip(sent_tags_2, json.loads(response.choices[0].message.content)+ ['_']*len(sentence_pos)):
    print(f'Original: {str(k):<30} Auto-Annotation: {m}')

Original: ['В', 'PREP']                  Auto-Annotation: ['В', 'PREP']
Original: ['России', 'NOUN']             Auto-Annotation: ['России', 'NOUN']
Original: ['вышла', 'VERB']              Auto-Annotation: ['вышла', 'VERB']
Original: ['первая', 'ADJF']             Auto-Annotation: ['первая', 'ADJF']
Original: ['практическая', 'ADJF']       Auto-Annotation: ['практическая', 'ADJF']
Original: ['книга', 'NOUN']              Auto-Annotation: ['книга', 'NOUN']
Original: ['«', 'PNCT']                  Auto-Annotation: ['«', 'PNCT']
Original: ['В', 'PREP']                  Auto-Annotation: ['В', 'NPRO']
Original: ['социальных', 'ADJF']         Auto-Annotation: ['социальных', 'ADJF']
Original: ['сетях', 'NOUN']              Auto-Annotation: ['сетях', 'NOUN']
Original: ['.', 'PNCT']                  Auto-Annotation: ['Twitter', 'NOUN']
Original: ['Twitter', 'LATN']            Auto-Annotation: ['—', 'PNCT']
Original: ['—', 'PNCT']                  Auto-Annotation: ['«', 'PNCT']
Original: ['«', 

Здесь уже есть проблема: модель все-таки забывает знаки препинаний. Попробуем разделить system prompt от user prompt

### Few-shot: system prompt

In [ ]:
SYSTEM_PROMPT = """Ты — система морфологической разметки русского текста. Используй только следующие части речи и сокращения:

```
NOUN — имя существительное

ADJF — имя прилагательное (полное)

ADJS — имя прилагательное (краткое)

COMP — компаратив

VERB — глагол (личная форма)

INFN — глагол (инфинитив)

PRTF — причастие (полное)

PRTS — причастие (краткое)

GRND — деепричастие

NUMR — числительное

ADVB — наречие

NPRO — местоимение-существительное

PRED — предикатив

PREP — предлог

CONJ — союз

PRCL — частица

INTJ — междометие

PNCT — знак пунктуации
```

Разметь каждое слово в предложении соответствующим тегом. Не добавляй других пояснений или описаний.

Формат вывода строго в json следующего формата:

[["слово", "тег"], ["слово", "тег"], ...]

Перед разметкой токенизируй предложение: помни, что каждый знак препинания – это отдельный токен.
"""

In [ ]:
USER_PROMPT = """Разметь следующее предложение: {sentence}."""

In [ ]:
prompt = USER_PROMPT.format(sentence=sent_2)
response = send_query(prompt, SYSTEM_PROMPT)

In [ ]:
for k, m in zip(sent_tags_2, json.loads(response.choices[0].message.content)+ ['_']*len(sent_tags_2)):
    print(f'Original: {str(k):<30} Auto-Annotation: {m}')

Original: ['В', 'PREP']                  Auto-Annotation: ['В', 'PREP']
Original: ['России', 'NOUN']             Auto-Annotation: ['Россия', 'NOUN']
Original: ['вышла', 'VERB']              Auto-Annotation: ['вышла', 'VERB']
Original: ['первая', 'ADJF']             Auto-Annotation: ['первая', 'ADJF']
Original: ['практическая', 'ADJF']       Auto-Annotation: ['практическая', 'ADJF']
Original: ['книга', 'NOUN']              Auto-Annotation: ['книга', 'NOUN']
Original: ['«', 'PNCT']                  Auto-Annotation: ['«', 'PNCT']
Original: ['В', 'PREP']                  Auto-Annotation: ['В', 'PREP']
Original: ['социальных', 'ADJF']         Auto-Annotation: ['социальных', 'ADJF']
Original: ['сетях', 'NOUN']              Auto-Annotation: ['сетях', 'NOUN']
Original: ['.', 'PNCT']                  Auto-Annotation: ['Twitter', 'NOUN']
Original: ['Twitter', 'LATN']            Auto-Annotation: ['—', 'PNCT']
Original: ['—', 'PNCT']                  Auto-Annotation: ['«', 'PNCT']
Original: ['«', 

### Few-shot

In [ ]:
SYSTEM_PROMPT = """Ты — система морфологической разметки русского текста. Используй только следующие части речи и сокращения:

```
NOUN — имя существительное

ADJF — имя прилагательное (полное)

ADJS — имя прилагательное (краткое)

COMP — компаратив

VERB — глагол (личная форма)

INFN — глагол (инфинитив)

PRTF — причастие (полное)

PRTS — причастие (краткое)

GRND — деепричастие

NUMR — числительное

ADVB — наречие

NPRO — местоимение-существительное

PRED — предикатив

PREP — предлог

CONJ — союз

PRCL — частица

INTJ — междометие

PNCT — знак пунктуации
```

Разметь каждое слово в предложении соответствующим тегом. Не добавляй других пояснений или описаний.

Формат вывода строго в json следующего формата:

[["слово", "тег"], ["слово", "тег"], ...]

Перед разметкой токенизируй предложение: помни, что каждый знак препинания – это отдельный токен.

Например:
{example_1}

{example_2}
"""

In [ ]:
text = 'В\tинститут\t—\tбез\tконкурса\t!'
tags = 'PREP\tNOUN\tPNCT\tPREP\tNOUN\tPNCT'
sent_tags_ex_1 = []
for word, tag in zip(text.split('\t'), tags.split('\t')):
    sent_tags_ex_1.append([word, tag])

text = 'Йоркширские\tдамы\t,\tможет\tбыть\t,\tтоже\tне\tпрочь\tбыли\tбы\tпредложить\tподобную\tнастойку\tсвоим\tгостям\t,\tно\tвоспитание\tне\tпозволяет\t.'
tags = 'ADJF\tNOUN\tPNCT\tVERB\tINFN\tPNCT\tPRCL\tPRCL\tADVB\tVERB\tPRCL\tINFN\tADJF\tNOUN\tADJF\tNOUN\tPNCT\tCONJ\tNOUN\tPRCL\tVERB\tPNCT'
sent_tags_ex_2 = []
for word, tag in zip(text, tags):
    sent_tags_ex_2.append([word, tag])

In [ ]:
system = SYSTEM_PROMPT.format(example_1=sent_tags_ex_1, example_2=sent_tags_ex_2)

In [ ]:
prompt = USER_PROMPT.format(sentence=sent_2)
response = send_query(prompt, system)

In [ ]:
for k, m in zip(sent_tags_2, json.loads(response.choices[0].message.content)+ ['_']*len(sent_tags_2)):
    print(f'Original: {str(k):<30} Auto-Annotation: {m}')

Original: ['В', 'PREP']                  Auto-Annotation: ['В', 'PREP']
Original: ['России', 'NOUN']             Auto-Annotation: ['России', 'NOUN']
Original: ['вышла', 'VERB']              Auto-Annotation: ['вышла', 'VERB']
Original: ['первая', 'ADJF']             Auto-Annotation: ['первая', 'ADJF']
Original: ['практическая', 'ADJF']       Auto-Annotation: ['практическая', 'ADJF']
Original: ['книга', 'NOUN']              Auto-Annotation: ['книга', 'NOUN']
Original: ['«', 'PNCT']                  Auto-Annotation: ['«', 'PNCT']
Original: ['В', 'PREP']                  Auto-Annotation: ['В', 'PREP']
Original: ['социальных', 'ADJF']         Auto-Annotation: ['социальных', 'ADJF']
Original: ['сетях', 'NOUN']              Auto-Annotation: ['сетях', 'NOUN']
Original: ['.', 'PNCT']                  Auto-Annotation: ['.', 'PNCT']
Original: ['Twitter', 'LATN']            Auto-Annotation: ['Twitter', 'NOUN']
Original: ['—', 'PNCT']                  Auto-Annotation: ['—', 'PNCT']
Original: ['«', 

## Выводы

Удивляет, что модель практически идеально справляется и на zero-shot (если ей дать список тегов). Однако даже с few-shot слабым местом почему-то остаются знаки препинания. Ей почему-то сложно токенизация дается, она их то теряет, то не замечает идущие подряд знаки. В принципе это кажется не сильно проблематичным, потому что можно сначала токенизировать, а потом подавать уже модели. С самой задачей пос-теггинга она справляется (на маленькой выборке из пары предложений) более чем хорошо.

# Задание 2


Проверьте серию GPT моделей на знание лингвистики. Найдите один хороший глубокий вопрос, с которым не справляется `gpt-4o`. Проверьте, как отвечают на этот вопрос более новые модели: `gpt-4.1`, `o3`, `gpt-5-mini`, `gpt-5`.


In [ ]:
question = 'Каков семантический тип квантора «каждый» в системе типов формальной семантики (Montague grammar)?'

In [ ]:
response = send_query(question, model_name='gpt-4o')

In [ ]:
print(response.choices[0].message.content)

В системе типов формальной семантики, как в грамматике Монтегю, квантор "каждый" (например, в выражении "каждый человек") представляется как выражение более высокого типа, чем простые именные группы. В частности, кванторы обычно интерпретируются как функции от предикатов к истинностным значениям.

Если предположить, что предикаты имеют тип \(\langle e, t \rangle\) (где \(e\) - это тип объектов, а \(t\) - это тип истинностных значений), то квантор "каждый" будет иметь тип \(\langle \langle e, t \rangle, t \rangle\). Это означает, что "каждый" принимает предикат в качестве аргумента и возвращает значение типа \(t\), то есть истина или ложь, в зависимости от того, выполняется ли предикат для всех объектов в области дискурса.


*В системе типов формальной семантики, как в грамматике Монтегю, квантор "каждый" (например, в выражении "каждый человек") представляется как выражение более высокого типа, чем простые именные группы. В частности, кванторы обычно интерпретируются как функции от предикатов к истинностным значениям.*

*Если предположить, что предикаты имеют тип ⟨e, t⟩⟩ (где \(e\) - это тип объектов, а \(t\) - это тип истинностных значений), то квантор "каждый" будет иметь тип ⟨⟨e, t⟩, t⟩. Это означает, что "каждый" принимает предикат в качестве аргумента и возвращает значение типа \(t\), то есть истина или ложь, в зависимости от того, выполняется ли предикат для всех объектов в области дискурса.*

---

И это неправильный ответ: слово «студент» имеет тип ⟨e, t⟩: у нас правда есть множество студентов –> студент принимает конкретного индивида и возвращает, является ли индивид студентом. В языке I порядка: Student(x) –> bool (X is a student / X is not a student); в лямбда исчислениях: $Λ x. x – student$
Поэтому и получается ⟨e, t⟩. Тогда для квантора точно будет другой типа. По сути квантор "каждый" интерпретируется как квантор типа ∀.
В лямбда исчислениях: Λ f: f ∈ D_(e, t). Λ g: g ∈ D_(e, t). ∀ x. f(x)=1 & g(x)=1.

По сути это двухаргументная функция, которая принимает типы ⟨e, t⟩, а возвращает булево значение t. Значит, квантор = ⟨⟨e, t⟩, ⟨⟨e, t⟩, t⟩⟩

То, что модель описала, – это кванторная именная группа. Она будет ⟨⟨e, t⟩, t⟩ (квантор "каждый" принимает "студент" –> позиция ⟨e, t⟩ заполняется –> остатеся⟨⟨e, t⟩, t⟩).



Можно попробовать дополнить вопрос, чтобы дать какие-то подсказки, которая модель может использовать и "поразмышляла".

In [ ]:
question = """Каков семантический тип квантора «каждый» в системе типов формальной семантики (Montague grammar)?
Перед ответом подумай: как сделать, чтобы не было конфликта типов при последующей деревации предложения"""

In [ ]:
response = send_query(question, model_name='gpt-4o')

In [ ]:
print(response.choices[0].message.content)

В системе типов формальной семантики Монтегю квантор "каждый" (например, в выражении "каждый человек") является функцией второго порядка. В этой системе кванторы анализируются как функции, которые принимают в качестве аргументов предикаты (функции типа \((e \rightarrow t)\)) и возвращают истину или ложь, то есть объект типа \(t\).

Чтобы избежать конфликта типов при деревации предложения, квантор "каждый" часто представляют как функцию, которая принимает предикат и возвращает утверждение о том, истинно ли, что данный предикат истинно для всех объектов домена. Таким образом, квантор "каждый" имеет тип \(((e \rightarrow t) \rightarrow t)\).

Пример:
- "Каждый человек спит": \( \text{в данном случае,} \) 
  - "каждый" принимает предикат "человек спит" типа \( (e \rightarrow t) \) и возвращает истинностное значение для всего утверждения.

Таким образом, используя типы и структуру системы формальной семантики Монтегю, мы можем обеспечить корректную типовую совместимость предложений, избегая

*- "каждый" принимает предикат "человек спит" типа \( (e → t) \) и возвращает истинностное значение для всего утверждения.*

Тут вообще нечто, противоречащее генеративизму. Квантор модифицирует все-таки именную группу, а не предложение. Да и предложение всегда имеет тип t. Предложение может быть либо истинным, либо ложным.

То есть модель молодец, что начала выворачиваться, но была надежда, что размышления ей помогут прийти к пониманию, где она была до этого неправа.


В общем мы нашли, с чем не справляется gpt-4o: подобно студенту примерно 3 курса бакалавриата теоретическое и прикладной лингвистики, который не особо заинтересован в математической логике и/или формальной семантике, модель не может вывести семантический тип верно.



---


Не используйте ключ сверх меры. Не создавайте циклов, который посылают множество однообразных запросов, отправляйте каждый запрос вручную. Не используйте очень длинные промпты (больше 2000 токенов).


---



In [ ]:
question = """Каков семантический тип квантора «каждый» в системе типов формальной семантики (Montague grammar)?"""

In [ ]:
GPT_4_1 = 'gpt-4.1'
O_3 = 'o3'
GPT_MINI = 'gpt-5-mini'
GPT_5 = 'gpt-5'

In [ ]:
response = send_query(question, model_name=GPT_4_1)

In [ ]:
print(response.choices[0].message.content)

В формальной семантике (грамматике Монтегю) семантический тип квантора "каждый" (“every”, “каждый”) — это **тип <<e,t>,<e,t>>** (читается как «от предиката к предикату»).

**Пояснение:**

- Объекты индивидуального типа имеют тип **e** (entity).
- Предикаты (например, глаголы-отношения ко множеству объектов) имеют тип **<e,t>** (от индивида к истинности).
- Кванторы — такие выражения, как "каждый", "все", "некоторые" — берут два предиката и возвращают значение истинности (t): один — ограничение области (restrictor), второй — свойства или предикат (scope).

В Монтеговской семантике:
- "каждый" (every) берет предикат **A** (например, student: <e,t>) и возвращает функцию, которая берет ещё один предикат **B** (например, smokes: <e,t>), и утверждает, что всякий, кто принадлежит **A**, также принадлежит **B**.

**Таким образом:**
- Семантический тип "каждый": <<e,t>,<e,t>>.

**Схематически:**
- "каждый" : <<e,t>,<e,t>>.
- every(A)(B) = true тогда и только тогда, когда для всех x, если A(x)=t

Это уже ближе, но все равно неверно. Денотат (выражение с лямбдами) написано верно: функция и вправда принимает два предиката и возвращает следование "если P(x), то Q(x)". Но она не учиывает, что это последнее выражение также отражается в семантическом типе. Попробуем подсказать.

In [ ]:
question_modified = """Каков семантический тип квантора «каждый» в системе типов формальной семантики (Montague grammar)?
Перед ответом подумай: как сделать, чтобы не было конфликта типов при последующей деревации предложения"""

In [ ]:
response = send_query(question, model_name=GPT_4_1)

In [ ]:
print(response.choices[0].message.content)

В формальной семантике типа Монтегю квантор "каждый" («every»/«каждый») выражается как выражение второго порядка, его семантический тип — ⟨⟨e, t⟩,⟨⟨e, t⟩, t⟩⟩.

### Объяснение

- **Тип e** обозначает индивидуумы (entities).
- **Тип t** — истиностные значения (truth-values).
- **Тип ⟨e, t⟩** — множество, свойство или предикат (например, функцию от индивидов к истине).
- **Тип ⟨⟨e, t⟩, t⟩** — множество свойств, к которым применён квантор (например, "все свойства, обладающие некоторым X").

Но квантор «каждый» принимает не просто имя (e), а два предиката (ограничитель и ядерный предикат), поэтому:

- Для выражения "Каждый студент читает книгу" ("Every student reads a book"):
    - «студент» — ⟨e, t⟩
    - «читает книгу» — ⟨e, t⟩
    - Квантор должен принимать два таких аргумента и возвращать t.

**То есть "каждый"**:  
Его семантический тип: **⟨⟨e, t⟩,⟨⟨e, t⟩, t⟩⟩**  
(функция от двух предикатов к истинностному значению).

**Иначе:**
- Сначала принимает ограничитель ("студент") — ⟨e, t⟩
-

```gpt-4.1``` теперь ответил верно.

> Семантический тип квантора «каждый» в грамматике Монтегю — ⟨⟨e, t⟩,⟨⟨e, t⟩, t⟩⟩.

In [ ]:
response = send_query(question, model_name=O_3)

In [ ]:
print(response.choices[0].message.content)

В формально-семантической системе Монтегю «каждый» (every/each) рассматривается как детерминатив, который

1) сначала «берёт» предикат-ограничитель P (напр. man) типа ⟨e,t⟩,
2) затем «берёт» предикат-ядеро Q (напр. walks) того же типа ⟨e,t⟩
3) и возвращает значение истинности.

Следовательно его семантический тип —

⟨⟨e,t⟩,⟨⟨e,t⟩,t⟩⟩   (чаще пишут <et,<et,t>>).

Семантическая абстракция:  
λP⟨e,t⟩. λQ⟨e,t⟩. ∀x (P(x) → Q(x)).


In [ ]:
response = send_query(question, model_name=GPT_MINI)

In [ ]:
print(response.choices[0].message.content)

Кратко: в Montagovской типовой системе «каждый» — детерминант типа <<e,t>,<<e,t>,t>> (или, в функционной записи, (e→t)→((e→t)→t)).

Лямбда‑запись стандартного смысла:
каждый = λP_{<e,t>}. λQ_{<e,t>}. ∀x (P(x) → Q(x)),

где P — предикат (существительное/ограничитель), Q — предикат сказуемого (вписываемый в область квантора). Таким образом после сочетания с существительным получается NP/DP типа <e,t>→t (обобщённый квантор), а после применения к VP — значение типа t.


In [ ]:
response = send_query(question, model_name=GPT_5)

In [ ]:
print(response.choices[0].message.content)

Квантор «каждый» (every) как детерминатор имеет тип <<e,t>,<<e,t>,t>>.

Интуиция: он берет предикат-ограничитель P типа <e,t> (например, «студент») и возвращает обобщенный квантор — функцию от предиката области Q типа <e,t> к истине t:
λP<e,t>. λQ<e,t>. ∀x (P(x) → Q(x)).

После применения к имени общего рода (<e,t>) результат — NP типа <<e,t>,t>.


``o3 | gpt-5-mini | gpt-5`` сразу ответили верно, даже грустно

# Задание 3

Проверьте тот же самый вопрос на любой из моделей Qwen 3 (например той что мы использовали на семинаре)

In [ ]:
question

'Каков семантический тип квантора «каждый» в системе типов формальной семантики (Montague grammar)?'

In [ ]:
import transformers, torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import userdata
HF_TOKEN = userdata.get('hf_general')

In [ ]:
model_id = "Qwen/Qwen3-4B-Thinking-2507"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id, token=HF_TOKEN)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_id, token=HF_TOKEN, dtype=torch.bfloat16, device_map="auto")

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [ ]:
model.device

device(type='cuda', index=0)

In [ ]:
text = tokenizer.apply_chat_template(
    [{"role": "user", "content": f"{question}"}],
    tokenize=False,
    add_generation_prompt=False,
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

In [ ]:
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=20000
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()

In [ ]:
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

In [ ]:
thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

In [ ]:
thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content) # no opening <think> tag

thinking content: a
0
0
<think>
Хорошо, мне нужно разобраться, какой семантический тип у квантора «каждый» в системе типов формальной семантики, то есть в монтажской грамматике. Давайте вспомню, что я знаю о типах в Монтаже.

Сначала вспомню, что в Монтаже типы определяют, как выражения взаимодействуют. Например, предложения имеют тип (e → t), где e — это тип сущностей (ентити), а t — тип истинности (булево). Но кванторы, наверное, имеют более сложные типы.

Кванторы в логике обычно обрабатывают кванторы по всему домену. В предикатной логике кванторы вроде ∀ (каждый) и ∃ (существует) имеют типы, которые зависят от типа, который они квантифицируют. Например, ∀x. P(x) имеет тип (e → t) для P(x), но в Монтаже типы могут быть более сложными.

В Монтаже типы задаются с помощью стрелок и букв. Например, типы могут быть: e (сущности), t (истина), (e → t) (предикаты), (e → e → t) (бинарные предикаты) и т.д.

Квантор «каждый» в английском — это квантор универсального количества, то есть ∀x. P(x

In [ ]:
print("content:", content)

content: Семантический тип квантора «каждый» в системе типов формальной семантики (Montague grammar) определяется как функция, которая принимает предикат, имеющий тип $ e \to t $, и возвращает тип истинности $ t $. 

В рамках Монтажевой грамматики:
- $ e $ — тип сущностей (например, люди, объекты),
- $ t $ — тип истинности (истина/ложь),
- $ e \to t $ — тип предикатов (свойств).

Квантор «каждый» (универсальный квантор) в Монтаже имеет тип $ (e \to t) \to t $, так как он квантифицирует по типу $ e $ и возвращает истину (или ложь) в результате применения к предикату.

Таким образом, семантический тип квантора «каждый» в системе типов формальной семантики (Montague grammar) — это:

$$
\boxed{(e \to t) \to t}
$$


**Ответ Qwen**: Семантический тип квантора «каждый» в системе типов формальной семантики (Montague grammar) определяется как функция, которая принимает предикат, имеющий тип $ e \to t $, и возвращает тип истинности $ t $.

В рамках Монтажевой грамматики:
- $ e $ — тип сущностей (например, люди, объекты),
- $ t $ — тип истинности (истина/ложь),
- $ e \to t $ — тип предикатов (свойств).

Квантор «каждый» (универсальный квантор) в Монтаже имеет тип $ (e \to t) \to t $, так как он квантифицирует по типу $ e $ и возвращает истину (или ложь) в результате применения к предикату.

Таким образом, семантический тип квантора «каждый» в системе типов формальной семантики (Montague grammar) — это:

$$
\boxed{(e \to t) \to t}
$$

Ну, и как уже до этого я объясняла, это неверный ответ. Тип, который написал Qwen, – тип кванторной именной группы, а не квантора.

# Задание 4

Выберите 3 языка, которые очень мало похожи на английский язык и которые мало похожи между собой. Используя токенизатор Qwen3 рассчитайте среднее отношение количества токенов к количеству символов для каждого языка. Для этого найдите какой-нибудь текст длинной более 5000 символов (или просто несколько текстов) и токенизируйте его. Поделите общее количество токенов на обшее количество символов.

In [ ]:
import re

In [ ]:
bulgarian = """И като се обърна по гърба си, той почна да нанася такива немилостиви удари с нозете си по повърхността на разпенената вода, щото бризги летяха чак до тавана. Той завъртя бързо ръце, за да изрази колелото на парахода: „Тупалупа, тупалупа, фйюууу“ и бай Ганьо изсвири с уста. Немците окаменяха на местата си. Те, по всяка вероятност, приеха моя другар за някой новодошъл, не постъпил още в лудницата възточен човек и аз забележих по лицата им не толкова негодувание, колкото съжаление. А бай Ганьо, вижда се, прочете по лицата им безкрайно удивление на неговото изкуство и затова възлезе бързешката по стълбата, изправи се малко разкрачен, изгледа високомерно немците, почна да се удря геройски по влакнатите гърди и извика победоносно: Булгар! Булгаар! и още посилно се удари в гърдите. Горделивият тон, с който изрече тази рекомендация, говореше много; този тон казваше: „Ето го, видите ли го българина! Този е той, такъв е той! Вий сте го чували само, сливнишкия герой, балканския гений! Ето го сега пред вас, цялцелиничък, от глава до пети, в натура! Видите ли какви чудеса е той в състояние да направи! И само това ли! Ехе, на какви работи още е той способен! Прости били българите, а! Гиди, чифути с чифути!“ А бе я попитай има ли сапун казва ми бай Ганьо, след като поизстина малко патриотическият му възторг, я виж нозете ми на какво ся заприличали…
И наистина, нозете му не представляваха найподходящ модел за Аполона Белведерски. Шарките на чорапите му бяха се отпечатали на кожата и без това нечиста и обрасла. Впрочем с нечистотата не можем зачуди българите: не можем накара и най разпалената фантазия да си представи нещо понечисто от туй, което може да ти представи самата действителност.
И аз имах една среща с бай Ганя обади се Кольо в Дрезден. Искате ли да ви я разкажа? Иска ли питане? Разказвай! извика цялата компания. Не зная дали сте чували преди няколко години дрезденското общество биде разтревожено от една крайно печална катастрофа, жертва на която стана една българка, ученичка, и един млад американец. Българката се учеше в Дрезден. В този град, както знаете, има цели квартали англичани, американци, руси и други иностранци, децата на които следват в тамошните учебни заведения. Една американка, ученичка, се сприятелява с нашата българка. Англичанките и американките ученички имат този каприз да вземат под своето покровителство някоя съученичка измежду найбеззащитните. За такава се е показала и нашата българка: посвитичка, посмиреничка, помълчаливичка. Сприятелили се те и американката почнала да я кани у дома си на гости по празнични дни. Българката се запознава с цялото им семейство, а между другите и с брата й, млад, двадесетгодишен момък, който изучаваше живопиството. Кой знае дали по свойствената на младите художници наклонност да се увличат от всякой жив модел, неподходящ под общия тип на окръжающите ги модели, дали по присъщата на американците склонност към оригиналност, но младият момък след няколко срещи с нашата скромна и срамежлива българка като че почна да сеща влечение към нея; той се ползуваше от всякой случай да се срещне с нея, да споделя своите артистически впечатления, да любезничи, с една дума, те почнаха да се любят. И тази любов им скъси дните. Един празничен ден през летните ваканции цялото семейство, придружено и от българката, тръгва на разходка из Саксонска Швейцария. При една скалиста живописна местност младият художник, увлечен в разговора с българката, се отдалечил от компанията. Една висока скала привлякла вниманието на момъка и той почнал да се катери по нея. Тръгва след него и спътницата му: но не успяват да стигнат и до средата на опасната скала, американецът се подхлъзва и в падането си увлича и българката в пропастта. Дотук им бил животът. Мъртвата двоица докараха в Дрезден, в клиниката. Цял град се разтревожи от това печално събитие. Сума свят се изпровървя да ги види, прострени един до друг върху една голяма мраморна маса. Веднага телеграфираха на брата на българката да дойде в Дрезден; поблизки роднини тя нямаше. Телата балсамираха, за да ги съхранят до пристигането му. Множество венци украсиха смъртното им легло. На третия ден се получи от Виена телеграма, че братът на покойната тръгва за Дрезден. Телата пренесоха в дома на американското семейство. Като очакваха с нетърпение пристигането на брата, сума мистери и мистриси, сума леди и джентлементи бяха се събрали в този дом. Семейството принадлежеше към висшето дрезденско общество. Бях отишъл там и аз с неколцина българи студенти. Имахме покани като българи, знайте, българкапокойница. Други пък българчета бяха отишли на станцията да посрещнат брата на покойната. Чакаме. Печална тържественост! Всичките апартаменти бяха пълни с посетители. Говореха тихо, с шепнене. Тишината се нарушаваше от време на време с горчивите ридания и отчаяни въздишки на убитата от скръб майка на покойния момък. В един интервал на мъртва тишина откъм коридора се зачу една страшна глъчка: чу се грубо тропане на обуща като че водеха подкован кон, чуха се гласове: „Къде е? Тука ли бе?“ „Шш!… Пополека!“ И в стаята на покойниците се вмъкна с калпак на главата… бай Ганьо."""

In [ ]:
len(bulgarian)

5107

In [ ]:
georgian = """აკაკიმ გიმნაზიიდან გამოსვლის შემდეგ ორი თვე სოფელში გაატარა. ამის შემდეგ სწავლის გასაგრძელებლად პეტერბურგში გაემგზავრა და იქ უნივერსიტეტში აღმოსავლური ენების ფაკულტეტზე ჩაირიცხა, რომელიც  წელს დაამთავრა. პეტერბურგის სახელი მჭიდროდ არის დაკავშირებული აკაკის პირად ცხოვრებასა და მის შემოქმედებასთან. ამ პერიოდში უნივერსიტეტში სწავლობდნენ და უფროსი კურსის სტუდენტებად ითვლებოდნენ: ილია ჭავჭავაძე, ნიკო ღოღობერიძე, ბესარიონ ღოღობერიძე და სხვები. აკაკისა და კირილე ლორთქიფანიძეს ჩასვლის მერე ქართველ სტუდენტთა დასს კიდევ მიემატნენ ნიკო ნიკოლაძე და გიორგი წერეთელი.
აკაკიმ ცოლად შეირთო ნატალია ბაზილევსკაია და ერთხანს მოსკოვში ცხოვრობდა, მაგრამ შემდეგ სამშობლოში დაბრუნდა, საცხოვრებელ ადგილად ქუთაისი აირჩია და ეროვნულ-განმანთავისუფლებელ მოძრაობაში ჩაება.
მიუხედავად მატერიალური ხელმოკლეობისა, აკაკი წერეთელი არასდროს შესულა სახელმწიფო სამსახურში.  წელს ქუთაისის საადგილმამულო ბანკის საზოგადო კრებამ აკაკის ერთხმად შესთავაზა ბანკის დირექტორის თანამდებობა. პოეტმა კატეგორიული უარი განაცხადა იმ დროისათვის ამ ერთგვარად საპატიო თანამდებობის დაკავებაზე, კრებას მადლობა გადაუხადა და თავისი უარი შემდეგნაირად დაასაბუთა: „მე რომ დირექტორობა ვიკისრო, საკუთრად მე მოხელეობას უნდა დავუდვა გული და ეგ იქნება ჩემთვის ბორკილი, თუმცა ოქროს ბორკილი ჩემთვის პირადად სარგებელია, მაგრამ მაინც ბორკილია და იმ საქმეში ხელის შემშლელი, რომელსაც მე ამდენი ხანია ვემსახურები“.
აკაკი წერეთელს უდიდესი დამსახურება მიუძღვის ''ქართველთა შორის წერა-კითხვის გამავრცელებელი საზოგადოების'' დაარსებასა და მის მრავალმხრივ კულტურულ-საგანმანათლებლო საქმიანობაში, ქართული დრამატული საზოგადოების შექმნასა და მუშაობაში, ქართული ჟურნალისტიკის განვითარებაში, როგორც უაღრესად ნაყოფიერი და მნიშვნელოვანი პუბლიცისტური მოღვაწეობით, ისე ყოველთვიური ჟურნალის "აკაკის თვიური კრებული" დაარსებით. იგი რედაქტორობდა აგრეთვე სატირულ-იუმორისტულ ჟურნალს "ხუმარა", რომლის ანტიცარისტული და ეროვნული მიმართულების გამო  წელს დაპატიმრებულიც კი იყო. ილია ჭავჭავაძესთან ერთად აკაკი წერეთელი სათავეში ჩაუდგა ეროვნულ-განმათავისუფლებელ მოძრაობას საქართველოში და სიტყვით თუ საქმით, დაუცხრომლად, მიზანდასახულად იღვწოდა ქართველი ხალხის სულიერი აღორძინებისათვის, მასში ეროვნული თვითშეგნების ამაღლებისათვის. სწორედ ეროვნული მოტივი იქცა აკაკი წერეთლის შემოქმედების ლაიტმოტივად.
სამოხელეო სამსახურზე უარის თქმით აკაკიმ საბოლოოდ მხოლოდ ქვეყნისა და ხალხის ინტერესების სამსახური არჩია. ეროვნული და სოციალური ჩაგვრის წინააღმდეგ ბრძოლაში აკაკი წერეთლისათვის მთავარი იარაღი მწერლობა იყო. იგი მთლიანად ლიტერატურულ ცხოვრებას მიეცა და თავისი შემოქმედებით აქტიურად ჩაება ხალხისა და სამშობლოს თავისუფლებისათვის ბრძოლაში.
ლექსების წერა აკაკი წერეთელმა ჯერ კიდევ ყრმობის ასაკში დაიწყო,  წელს იგი უკვე რამდენიმე დაბეჭდილი ლექსის ავტორი იყო, ხოლო  წელს გამოქვეყნებულმა ლირიკულმა ლექსმა  "საიდუმლო ბარათი", რომელიც იმთავითვე სიმღერადაც გავრცელდა, ახალგაზრდა პოეტს ფართო პოპულარობა მოუტანა. ეროვნული თვითშეგნების აღორძინებაში დიდი როლი შეასრულა აკაკი წერეთლის ისტორიულმა პოემებმა და დრამებმა: "ბაგრატ დიდი", "თორნიკე ერისთავი", "პატარა კახი", რომანმა "ბაში-აჩუკი" და სხვა. უდიდესი აღმზრდელობითი მნიშვნელობა ჰქონდა პოემას "გამზრდელი", რომელშიც ადამიანური ზნეობის პრობლემებია წამოჭრილი.
აკაკი წერეთლის შემოქმედებამ წარუშლელი კვალი დატოვა ქართველი ხალხის სულიერი კულტურის ისტორიაში და მან კანონიერად დაიმკვიდრა ახალი ქართული ლიტერატურისა და სალიტერატურო ენის რეფორმატორისა და ფუძემდებლის სახელი.
აკაკი წერეთელმა ახალი ქართული სალიტერატურო ენის განვითარების, სრულყოფისა და ხალხში მისი დამკვიდრების დიდ სახალხო საქმეში მეტად ნაყოფიერი და მნიშვნელოვანი როლი შეასრულა. უაღრესად დიდი ისტორიული მნიშვნელობა ჰქონდა ქართული სალიტერატურო ენის ბუნებრივი განვითარებისათვის თავდადებულ ბრძოლას და დაუღალავ ზრუნვას ხალხისათვის გასაგები, საყოველთაო ეროვნული ლიტერატურული ენის გასანვითარებლად და ხალხში დასამკვიდრებლად. ამ მისიის შესრულება იტვირთეს ქართველმა სამოციანელებმა: ილია ჭავჭავაძემ, აკაკი წერეთელმა, იაკობ გოგებაშვილმა, ნიკო ნიკოლაძემ, გიორგი წერეთელმა, კირილე ლორთქიფანიძემ და სხვებმა.
აკაკიმ და ილიამ გადამწყვეტი იერიშები მიიტანეს ლიტერატურულ ენაში არქაული სტილისა და დიალექტების გაბატონების წინააღმდეგ და თავისი მდიდარი შემოქმედებითი პრაქტიკით გზა გაუკაფეს ერთიანი სახალხო-ლიტერატურული ენის დამკვიდრებას, მის ბუნებრივ განვითარებას. აკაკი იმთავითვე ხალხისათვის გასაგები ენით წერდა.
აკაკი წერეთელი ლიტერატურულ-შემოქმედებით მუშაობასთან ერთად უდიდეს მნიშვნელობას ანიჭებდა სკოლების, თეატრების, ბიბლიოთეკა-სამკითხველოებისა და სხვა სახის კულტურულ-საგანმანათლებლო დაწესებულებათა განვითარებას. აკაკიმ დიდი მუშაობა ჩაატარა ახალი ქართული რეალისტური თეატრის აღორძინებისა და განვითარებისათვის. ქუთაისში ცხოვრების პერიოდში იგი ახლო გაეცნო ცნობილ პოეტს რაფიელ ერისთავს, რომელიც იმ დროს იქ მომრიგებელ მოსამართლედ მსახურობდა, მასთან ერთად ჩამოაყალიბა დრამატული წრე და შეუდგა წარმოდგენებისა და ლიტერატურული საღამოების მოწყობას. „სახელმწიფო სახლში“ გამართეს სცენა და კვირაში ერთხელ, აგრეთვე დღესასწულებზე, აწყობდნენ წარმოდგენებს. წარმოდგენების შემოსავლით ინახავდნენ „ქალთა უსასყიდლო სკოლას“.
აკაკი ქუთაისში გამართულ თეატრულ წარმოდგენებში მონაწილეობდა როგორც რეჟისორი და აქტიორი. აკაკი მაღალ შეფასებას აძლევდა მსახიობს, მის მოქალაქეობრივ როლს საზოგადოებაში. იგი წერდა: „აქტიორი იგივე მოძღვარია ხალხისა. ის არის მწერლის თანაშემწე… მწერლის სურათებს ის ახორციელებს და ერთად თავმოყრილ ხალხს ცოცხლად და ნათლად გადასცემს."
თბილისსა და ქუთაისს გარდა აკაკი წარმოდგენებს მართავდა სხვადასხვა ქალაქებში, რითაც ხელს უწყობდა ფართო მასებში თეატრალური კულტურის შეტანას და მოწინავე იდეების დანერგვას.
"""

In [ ]:
len(georgian)

5364

In [ ]:
finnish = """Waltarin synnyinkoti oli Helsingin Siltasaaressa Saariniemenkatu :ssa olleessa talossa. Kuvanveistäjä Jouko Toiviaisen suunnittelemassa muistolaatassa nykyisen talon seinässä lukee: ”Tällä paikalla sijainneessa talossa syntyi kirjailija akateemikko Mika Waltari”, ja siinä on myös Waltarin kasvokuva. Waltarin pappisisä Toimi Waltari kuoli pojan ollessa viisivuotias, jolloin kasvatusvastuu jäi äidille. Mikalla oli kaksi veljeä: Samuel (kutsumanimeltä Samuli) ja Erkki. Samuli oli Mikaa kaksi vuotta vanhempi ja Erkki viisi vuotta nuorempi. Mikan isä Toimi ja setä Toivo Waltari olivat molemmat teologeja ja julkaisivat lukuisia uskonnollisia kirjoja. Kirjat julkaisi useimmiten Suomen Merimieslähetys. Waltari kirjoitti  ylioppilaaksi Helsingin suomalaisen normaalilyseon klassiselta linjalta. Hän oli koulun pilalehden Pillerin toimittaja. Koulun jälkeen Waltari aloitti teologian opinnot Helsingin yliopistossa mutta siirtyi elämänkatsomuksellisen kriisin jälkeen pian filosofisen tiedekunnan historiallis-kielitieteelliseen osastoon. Filosofian kandidaatiksi hän valmistui . Hänen käytännöllisen filosofian pro gradu -tutkielmansa Taivaallinen ja maallinen rakkaus käsitteli uskonnon ja erotiikan välistä suhdetta. Kirjallisen uran Waltari aloitti -vuotiaana Merimieslähetyksen tilaamalla uskonnollisella kertomuksella Jumalaa paossa. Waltari oli mukana myös Tulenkantajien piirissä ja ehti ennen läpimurtoteostaan Suuri illusioni kirjoittaa runoja ja salanimellä Kristian Korppi kauhunovellikokoelman Kuolleen silmät. Hän piti aluksi itseään nimenomaan runoilijana. Runoilijaryhmänä aloittanut, varsin lyhytaikainen Tulenkantajat ei ollut ohjelmallisesti yhtenäinen ryhmä. Yhteistä siihen kuuluneille oli vain iloinen, vapautunut perusmieliala, luottamus ihmisyyteen ja nuoren tasavallan sekä suomalaisen kulttuurin tulevaisuuteen ja kiinnostus kirjallisuuden ja tekniikan alan uusiin ilmiöihin. Runosta proosaan siirtyneen Waltarin Suuri illusioni ilmensi näitä tunnelmia, ja teoksesta tuli -luvun kulttiromaani. Toisin kuin muut tulenkantajat, Waltari pystyi nopeasti sopeutumaan enemmistöön ja kirjoittamaan sekä viihdettä että taidetta. Tammikuussa  Mika Waltari matkusti Rivieralle, jossa oleskeli Nizzassa, Mentonissa ja Monte Carlossa. Helmikuun alussa hän jatkoi matkaansa Pariisiin ja viipyi siellä maaliskuun loppupuolelle asti. Matka auttoi Waltarin eroon kiusallisesta tuttavuudesta Minna Craucheriin. Vuonna  murhattu Craucher oli esikuvana Suuren illusionin madame Spindelille. Armeija-aika Waltari aloitti varusmiespalveluksensa  Uudenmaan rykmentissä Santahaminassa, jonne tuolloin kuljettiin meritse. Jo palvelukseen astuessaan Waltari oli kuuluisuus. Hänet määrättiin aliupseerikoulutukseen Parolaan. Reserviupseerikoulussa hän tunsi tiettyä ”aateluuden tuntoa” vastuun lisääntymisen myötä. Upseerikoulutuksessa hän joutui auto-onnettomuuteen. RUK:n kuljetusauton tieltä suistumisessa hän loukkasi päänsä, ja hänet vapautettiin kesken palveluksen. Toinen syy vapautukseen oli hänen sairastamansa kuiva keuhkopussitulehdus. Waltari kirjoitti asevelvollisuusajastaan armeijamyönteisen omaelämäkerrallisen teoksen Siellä missä miehiä tehdään vielä Tilkassa toipuessaan. Toisaalta isänsä ystävälle Jalo Sihtolalle lähettämässään kirjeessä Waltari tekee selväksi, ettei viihdy armeijassa. Waltari-tutkija Panu Rajala on arvellut, että Waltarin yksi vaikutin armeijamyönteisen teoksen tekemiseen oli, että hänen morsiamensa Marjatta Luukkosen isä oli lääkintäkenraalimajuri Emil Luukkonen ja Waltari halusi miellyttää tulevaa appeaan. Toimittaja ja ammattikirjailija Waltari meni  naimisiin Marjatta Luukkosen kanssa. Perheeseen syntyi  tytär Satu. Perheensä elättämiseksi Waltari omaksui ammattikirjoittajan työmoraalin. Hänellä oli vakinainen työ Suomen Kuvalehden toimittajana vuoteen  saakka, jolloin hän jäi vapaaksi kirjailijaksi. -luvulla hän kirjoitti romaaneja, novelleja, satuja, runoja ja  näytelmää. Vuosikymmen olikin hänen uutterimpia kirjoittajarupeamiaan, minkä osaltaan mahdollisti opintojen loppuun vieminen jo aiemmin. Sotien aikana Waltari työskenteli Valtion tiedoituslaitoksessa. Hän osallistui sotapropagandan tekoon muun muassa kirjoittamalla tietokirjat Neuvostovakoilun varjossa, Totuus Virosta, Latviasta ja Liettuasta sekä romaanit Antero ei enää palaa ja Rakkaus vainoaikaan. Hän kirjoitti paljon lehtiartikkeleita, isänmaallisia reportaaseja rintamalta ja sanomalehtien pääkirjoituksia. Waltari osallistui lokakuussa  suomalaisten kirjailijoiden vierailuun Weimarin kirjailijakokoukseen, joka pidettiin propagandaministeri Joseph Goebbelsin suojeluksessa. Syksyllä  hän aloitti Istanbuliin suuntautuneen matkan, jonka aikana hän poikkesi vajaaksi viikoksi Sveitsiin ja tapasi siellä myös Jean-Louis Perret’n, jonka Sinuhe-ranskannoksen ensimmäinen osa ilmestyi – ja nosti Sinuhen maailmanmaineeseen. Mika Waltari oli erittäin tuottelias kirjailija, jonka tuotantoon kuuluu yli romaania, viitisentoista pienoisromaania, 34 näytelmää ja lisäksi kuunnelmia, pieniä dialogeja, elokuvakäsikirjoituksia, runoja, matkakertomuksia, novelleja ja lukuisia muita tekstejä. Hänen ensimmäinen julkaistu teoksensa oli Jumalaa paossa vuonna, ja viimeiset tekstit hän saneli Ihmisen ääni -sarjassa ilmestyneeseen teokseen vuonna. Hänen teoksistaan on otettu jatkuvasti lisäpainoksia ja niitä on julkaistu myös erilaisina kokoelmina; esimerkiksi hänen näytelmänsä julkaistiin yhtenä kokoelmana vasta."""

In [ ]:
len(finnish)

5431

In [ ]:
bulgarian_tokenization = tokenizer([bulgarian], return_tensors="pt").to(model.device)
georgian_tokenization = tokenizer([georgian], return_tensors="pt").to(model.device)
finnish_tokenization = tokenizer([finnish], return_tensors="pt").to(model.device)

In [ ]:
print(f"Отношение токенов к символам в болгарском: {len(bulgarian_tokenization['input_ids'][0]) / len(bulgarian):.3f}")
print(f"Отношение токенов к символам в грузинском: {len(georgian_tokenization['input_ids'][0]) / len(georgian):.3f}")
print(f"Отношение токенов к символам в финнском: {len(finnish_tokenization['input_ids'][0]) / len(bulgarian):.3f}")

Отношение токенов к символам в болгарском: 0.446
Отношение токенов к символам в грузинском: 0.995
Отношение токенов к символам в финнском: 0.395


Очевидно, что из-за нестандартной графики, для грузинского отношение стремится к 1 -> одном символу соответствует один токен.